In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [11]:
v = 'v3'
final_clusters = pd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/final_clusters_{v}.pq')[0]
from core.cluster_validation import get_color
final_colors = pd.DataFrame(get_color(final_clusters.values), final_clusters.values).drop_duplicates()
final_colors.loc[-1] = [255,255,255]

In [17]:
regions = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')


In [ ]:
regions.explore()

In [13]:


region_id = 69333  # prague

# region_id = 99886 # bratislava

# region_id = 151676 # vilnius


# region_id = 8707 # frankfurt / mainz
# region_id = 5883 # freiburtg
# region_id = 38679 # munich
# region_id = 55763 # berlin
# region_id = 34266 # nuremberg

# region_id = 86873 # vienna


# region_id = 107131 # krakow

# region_id= 66593 # usti

# region_id = 62954 # lindz
# region_id = 91011 # above brno


In [14]:
etcs = gpd.read_parquet(f"/data/uscuni-ulce/processed_data/clusters/{region_id}_clusters.pq")

In [15]:
import lonboard
layer = lonboard.SolidPolygonLayer.from_geopandas(etcs, opacity=.7)

from sidecar import Sidecar
sc = Sidecar(title=f'Final Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.DarkMatter)
with sc:
    display(m)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [16]:
from core.cluster_validation import get_color
layer.get_fill_color = final_colors.loc[etcs.final].values.astype('uint8')

In [10]:
etcs.final.nunique()

29

In [9]:
# layer.get_fill_color = get_color(etcs.morph.factorize()[0])

In [43]:
# etcs[etcs.final == 27].explore()

In [20]:
cdata = pd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/cdata_{v}.pq')

In [21]:
cdata.style.background_gradient(axis=0, cmap="BuGn")

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,mtbSWR,libNCo,ldbPWL,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbSAl,sdsLen,sssLin,ldsMSL,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,sddAre,midRea,midAre,sdcLAL,sdcAre,sscCCo,sscERI,mtcWNe,mdcAre,ltcWRB,sicCAR,sdsAre,mibCou,mibAre,mibLen,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,midBAD,limLPS,limLPS2
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,59.035621,33.319189,0.000013,0.545292,4.000000,0.289341,0.998329,0.684213,0.251475,0.000000,73.481722,0.653571,1.810581,8.116889,10.368464,4.185243,159.059980,0.995012,130.670597,413.000000,259279.166363,0.185599,22.000674,0.361457,4.509304,3.000000,0.128525,0.731991,0.121212,0.127913,0.000018,0.005785,0.010928,119.932057,0.076923,125.714727,13918.806999,88.000000,43965.744070,29.439065,285.743846,0.451340,0.965497,0.090093,2794.042806,0.000157,0.232058,11694.016018,3.000000,213.788845,73.078886,0.657044,0.839128,0.442513,24.842727,2.847808,2.847808,57.138336,61.845816,1.000000,1.000000
1,59.826591,33.373752,0.000012,0.540799,4.000000,0.296467,0.998600,0.674418,0.246783,0.000000,75.968714,0.666667,2.290672,8.612329,11.258128,5.546551,161.829306,0.990118,136.483845,344.000000,250329.387986,0.185989,22.622030,0.388060,4.572150,3.000000,0.096787,0.719030,0.093750,0.173077,0.000014,0.005771,0.010168,207.744047,0.000000,130.418454,14681.501673,68.000000,39119.057153,31.232415,318.303776,0.445996,0.964489,0.085157,3183.712532,0.000110,0.222728,14258.709457,3.000000,224.303837,75.617305,0.646154,0.844877,0.434557,25.701796,2.885996,2.885996,58.713769,63.894214,1.000000,1.000000
2,78.507328,37.953673,0.000000,0.542370,4.000000,0.480136,0.996288,0.680234,0.219551,0.000000,76.539843,0.666667,2.770419,8.793370,11.451593,5.297853,146.613329,0.993243,125.987791,309.500000,234696.113544,0.159993,21.984199,0.387960,4.726763,3.000000,0.109589,0.743590,0.095346,0.147059,0.000016,0.006039,0.010846,165.809090,0.000000,118.602107,12670.411350,59.500000,35455.450473,32.419769,357.324652,0.457661,0.966346,0.081297,3430.184974,0.000124,0.231585,11777.718883,2.750000,231.365912,76.080162,0.632445,0.844165,0.433571,26.198196,2.968660,2.968660,65.171902,70.108925,1.000000,1.000000
3,74.616407,37.048553,0.000000,0.542375,4.000000,0.689916,0.998890,0.665509,0.217411,0.000000,82.914016,0.666667,3.160341,7.686398,10.899668,6.666085,198.147921,0.986800,225.657121,419.000000,454853.196846,0.199953,21.666305,0.396714,5.334130,3.000000,0.084337,0.725524,0.071429,0.195238,0.000009,0.003372,0.005569,338.991206,0.000000,158.196469,22316.208810,88.000000,69393.106126,30.957669,335.801418,0.462460,0.967124,0.084226,3211.820088,0.000101,0.242061,21023.165135,3.000000,246.314784,82.878065,0.634337,0.813748,0.412361,28.366602,2.943683,2.943683,62.364201,72.795813,1.000000,1.000000
4,59.479066,33.038864,0.000000,0.564301,4.000000,0.226077,0.999403,0.723789,0.168046,0.000000,56.715404,0.750000,1.568818,9.574577,11.513573,3.610712,151.535210,0.996363,124.330241,322.000000,216655.815875,0.160965,23.136530,0.407208,4.354644,3.000000,0.130435,0.750000,0.109971,0.127334,0.000013,0.005946,0.011073,96.707376,0.083916,119.981598,12369.221927,71.000000,37511.245157,31.485151,343.351255,0.451906,0.966152,0.084603,3127.461461,0.000150,0.193319,10841.939385,2.000000,152.854308,56.015459,0.694175,0.917858,0.502763,19.547866,2.627163,2.627163,52.448704,54.895644,1.000000,1.000000
5,69.985129,35.930759,0.000000,0.536859,4.000000,0.395135,0.998274,0.657388,0.290155,0.000000,92.287927,0.600000,1.307488,8.256457,10.784308,3.224458,145.374339,0.997742,124.098551,353.000000,235640.029783,0.181582,21.983567,0.366484,4.200372,3.000000,0.136986,0.742857,0.128264,0.116279,0.000013,0.005973,0.011443,94.494534,0.076923,117.692775,12201.451714,75.000000,38745.945258,30.219596,274.660740,0.425648,0.969263,0.088416,2693.367321,0.000158,0.258418,10442.417453,4.000000,295.519230,91.952198,0.562714,0.826885,0.387273,3

In [23]:
cdata.index[cdata['mibAre'] < 150]

Index([6, 12, 13, 15, 17, 24, 28, 31], dtype='int64', name='0')

In [9]:
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
region_hulls.explore()